In [2]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 15.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.48.0 to work with demo-dp100


In [3]:
# creating a new folder for it
import os, shutil

# Create a folder for the experiment files
folder_name = 'churn-experiment-files'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/telco.xlsx', os.path.join(folder_name, "telco.xlsx"))

'churn-experiment-files/telco.xlsx'

**Run the experiment as script**

In [4]:
%%writefile $folder_name/churn_exp.py
from azureml.core import Run
import pandas as pd 
import os 


# Get the experiment run context
run = Run.get_context()

#Load the dataset
data = pd.read_excel('telco.xlsx',engine='openpyxl')

# Count the rows and log the result
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# Count and log the label counts
churn_counts = data['churn'].value_counts()
print(churn_counts)
for k, v in churn_counts.items():
    run.log('Label:' + str(k), v)
      
# Save a sample of the data in the outputs folder (which gets uploaded automatically)
os.makedirs('outputs', exist_ok=True)
data.sample(100).to_csv("outputs/sample.csv", index=False, header=True)

# Complete the run
run.complete()

Overwriting churn-experiment-files/churn_exp.py


In [18]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails

# Create a Python environment for the experiment
#env = Environment('demodp100-env')

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='churn_exp.py',
                                environment=experiment_env,
                                docker_runtime_config=DockerConfiguration(use_docker=True))

# submit the experiment
experiment = Experiment(workspace=ws, name='mslearn-churn')
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [5]:
%%writefile $experiment_folder/experiment_env.yml
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- seaborn
- scipy
- numpy
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Writing ./churn-experiment-files/experiment_env.yml


In [6]:
from azureml.core import Environment

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/experiment_env.yml")

# Let Azure ML manage dependencies
experiment_env.python.user_managed_dependencies = False 

# Print the environment details
print(experiment_env.name, 'defined.')
print(experiment_env.python.conda_dependencies.serialize_to_string())

experiment_env defined.
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- seaborn
- scipy
- numpy
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow



In [9]:
# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

workspacefilestore - Default = False
workspaceworkingdirectory - Default = False
workspaceartifactstore - Default = False
workspaceblobstore - Default = True


In [12]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

Dataset.File.upload_directory(src_dir='data',
                              target=DataPath(default_ds, 'chrun-data/')
                              )

Validating arguments.
Arguments validated.
Uploading file to chrun-data/
Uploading an estimated of 3 files
Uploading data/.amlignore
Uploaded data/.amlignore, 1 files out of an estimated total of 3
Uploading data/.amlignore.amltmp
Uploaded data/.amlignore.amltmp, 2 files out of an estimated total of 3
Uploading data/telco.xlsx
Uploaded data/telco.xlsx, 3 files out of an estimated total of 3
Uploaded 3 files
Creating new dataset


{
  "source": [
    "('workspaceblobstore', '/chrun-data/')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ]
}

In [15]:
from azureml.core import Dataset

# Get the default datastore
default_ds = ws.get_default_datastore()

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'chrun-data/telco.xlsx'))
tab_data_set = Dataset.Tabular.f

# Display the first 20 rows as a Pandas dataframe
tab_data_set.take(20).to_pandas_dataframe()

DatasetValidationError: DatasetValidationError:
	Message: Failed to validate the data.
ScriptExecutionException was caused by StreamAccessException.
  StreamAccessException was caused by NotFoundException.
    Found no resources for the input provided: '[REDACTED]'
| session_id=b98b6169-aad9-47da-b752-664607f7c6f0
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Failed to validate the data.\nScriptExecutionException was caused by StreamAccessException.\n  StreamAccessException was caused by NotFoundException.\n    Found no resources for the input provided: '[REDACTED]'\n| session_id=b98b6169-aad9-47da-b752-664607f7c6f0"
    }
}